In [ ]:
import os
import numpy as np
import pandas as pd
import fastf1
from tqdm import tqdm
from typing import Dict, Any, List


# ================
# CONFIG
# ================
fastf1.Cache.enable_cache("cache")     # Change path if needed

SEASONS = [2023]                       # Add more seasons if needed
SESSION_TYPES = ["R"]                  # Only Race sessions (FP1/FP2/Q later if needed)
OUTPUT_PATH = "data/f1_ml_laps_dataset.csv"


# ================
# UTILITIES
# ================

def safe_timedelta_to_sec(val):
    """Convert pandas Timedelta/NaT → seconds float."""
    if pd.isna(val):
        return np.nan
    return float(val.total_seconds())


def extract_telemetry_features(lap) -> Dict[str, Any]:
    """
    Basic telemetry summaries only.
    FastF1 provides Speed, Throttle, Brake, Gear, RPM, DRS.
    """
    features = {
        "speed_mean": np.nan,
        "speed_max": np.nan,
        "throttle_mean": np.nan,
        "brake_mean": np.nan,
        "rpm_mean": np.nan,
        "rpm_max": np.nan,
        "drs_activations": np.nan,
    }

    try:
        tel = lap.get_car_data().add_distance()
    except Exception:
        return features

    if tel.empty:
        return features

    # Speed
    if "Speed" in tel.columns:
        features["speed_mean"] = float(tel["Speed"].mean())
        features["speed_max"] = float(tel["Speed"].max())

    # Throttle
    if "Throttle" in tel.columns:
        features["throttle_mean"] = float(tel["Throttle"].mean())

    # Brake
    if "Brake" in tel.columns:
        features["brake_mean"] = float(tel["Brake"].mean())

    # RPM
    if "RPM" in tel.columns:
        features["rpm_mean"] = float(tel["RPM"].mean())
        features["rpm_max"] = float(tel["RPM"].max())

    # DRS
    if "DRS" in tel.columns:
        features["drs_activations"] = int((tel["DRS"] > 0).sum())

    return features


def extract_weather_for_lap(lap, weather_df: pd.DataFrame) -> Dict[str, Any]:
    """Return nearest weather sample to lap midpoint."""
    default = {
        "air_temp": np.nan,
        "track_temp": np.nan,
        "humidity": np.nan,
        "wind_speed": np.nan,
        "wind_dir": np.nan,
        "pressure": np.nan,
    }

    if weather_df is None or weather_df.empty:
        return default

    lap_start = lap.get("LapStartTime")
    lap_time = lap.get("LapTime")

    if pd.isna(lap_start) or pd.isna(lap_time):
        return default

    try:
        lap_mid = lap_start + (lap_time / 2)
        idx = (weather_df["Time"] - lap_mid).abs().idxmin()
        w = weather_df.loc[idx]

        return {
            "air_temp": float(w.get("AirTemp", np.nan)),
            "track_temp": float(w.get("TrackTemp", np.nan)),
            "humidity": float(w.get("Humidity", np.nan)),
            "wind_speed": float(w.get("WindSpeed", np.nan)),
            "wind_dir": float(w.get("WindDirection", np.nan)),
            "pressure": float(w.get("Pressure", np.nan)),
        }
    except Exception:
        return default


def extract_lap_row(lap, session) -> Dict[str, Any]:
    """Combine lap, tyre, telemetry, and weather into one training row."""
    tel_feats = extract_telemetry_features(lap)
    weather_feats = extract_weather_for_lap(lap, session.weather_data)

    pit_in = lap.get("PitInTime")
    pit_out = lap.get("PitOutTime")
    is_pit = (not pd.isna(pit_in)) or (not pd.isna(pit_out))

    return {
        # Meta
        "season": session.event.get("Year", np.nan),
        "gp_name": session.event.get("EventName", "na"),
        "session_name": session.name,
        "session_type": session.session_info.get("Type", "NA"),

        "driver": lap.get("Driver", "na"),
        "team": lap.get("Team", "na"),

        # Lap info
        "lap_number": lap.get("LapNumber", np.nan),
        "lap_time": safe_timedelta_to_sec(lap.get("LapTime")),
        "sector_1_time": safe_timedelta_to_sec(lap.get("Sector1Time")),
        "sector_2_time": safe_timedelta_to_sec(lap.get("Sector2Time")),
        "sector_3_time": safe_timedelta_to_sec(lap.get("Sector3Time")),
        "position": lap.get("Position", np.nan),
        "track_status": lap.get("TrackStatus", np.nan),
        "is_pit_lap": is_pit,

        # Tyres
        "compound": lap.get("Compound", "na"),
        "stint": lap.get("Stint", np.nan),
        "tyre_life": lap.get("TyreLife", np.nan),
        "fresh_tyre": lap.get("FreshTyre", np.nan),

        # Basic telemetry
        **tel_feats,

        # Weather
        **weather_feats,
    }


def get_event_names_for_season(season: int) -> List[str]:
    """Filter out any testing sessions automatically."""
    schedule = fastf1.get_event_schedule(season)

    schedule["EventFormat"] = schedule["EventFormat"].astype(str).str.lower()
    schedule = schedule[~schedule["EventFormat"].str.contains("testing")]

    return schedule["EventName"].tolist()


# ================
# MAIN PIPELINE
# ================

def main():
    all_rows: List[Dict[str, Any]] = []

    for season in SEASONS:
        gp_names = get_event_names_for_season(season)

        for gp in gp_names:
            for sess_type in SESSION_TYPES:

                label = f"{season} - {gp} - {sess_type}"
                print(f"\n📂 Loading session: {label}")

                try:
                    session = fastf1.get_session(season, gp, sess_type)
                    session.load()
                except Exception as e:
                    print(f"❌ Failed to load {label}: {e}")
                    continue

                laps = session.laps
                if laps is None or laps.empty:
                    print(f"⚠️ No laps found for {label}")
                    continue

                # Keep only proper timed laps
                laps = laps[~laps["LapTime"].isna()]

                # Build rows
                for _, lap in tqdm(laps.iterrows(), total=len(laps),
                                   desc=f"   🏎 Processing laps for {gp}"):
                    try:
                        all_rows.append(extract_lap_row(lap, session))
                    except Exception as e:
                        print(f"⚠️ Skipped lap {lap.get('LapNumber', 'na')}: {e}")

    if not all_rows:
        print("\n⚠️ No data extracted.")
        return

    df = pd.DataFrame(all_rows)
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

    df.to_csv(OUTPUT_PATH, index=False)
    print(f"\n✅ Saved {len(df)} laps → {OUTPUT_PATH}")
    print(f"   Columns: {df.columns.tolist()}")


if __name__ == "__main__":
    main()


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...



📂 Loading session: 2023 - Bahrain Grand Prix - R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
   🏎 Processing laps for Bahrain Grand Prix: 100%|██████████| 1055/1055 [00:12<00:00, 85.51it/s]
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core 


📂 Loading session: 2023 - Saudi Arabian Grand Prix - R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 11 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
   🏎 Processing laps for Saudi Arabian Grand Prix: 100%|██████████| 924/924 [00:10<00:00, 89.54it/s] 
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            

...


📂 Loading session: 2023 - Las Vegas Grand Prix - R


req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:00.001000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', '23', '20', '3', '24', '2', '77', '22', '27', '4']
   🏎 Processing laps for Las Vegas Grand Prix: 100%|██████████| 861/861 [00:18<00:00, 45.84it/s]
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using


📂 Loading session: 2023 - Abu Dhabi Grand Prix - R


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']
   🏎 Processing laps for Abu Dhabi Grand Prix: 100%|██████████| 1157/1157 [00:27<00:00, 42.52it/s]



✅ Saved 23983 laps → data/f1_ml_laps_dataset.csv
   Columns: ['season', 'gp_name', 'session_name', 'session_type', 'driver', 'team', 'lap_number', 'lap_time', 'sector_1_time', 'sector_2_time', 'sector_3_time', 'position', 'track_status', 'is_pit_lap', 'compound', 'stint', 'tyre_life', 'fresh_tyre', 'speed_mean', 'speed_max', 'throttle_mean', 'brake_mean', 'rpm_mean', 'rpm_max', 'drs_activations', 'air_temp', 'track_temp', 'humidity', 'wind_speed', 'wind_dir', 'pressure']
